<a href="https://colab.research.google.com/github/stefsoliveira/iaad/blob/main/%5BIAAD%5DIntelicipes_Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

## Import libraries

In [418]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importing the dataset from the google drive

In [419]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing the tables

In [420]:
recipe = pd.read_csv('/content/drive/My Drive/InteliCipes/RAW_recipes.csv')
interaction = pd.read_csv('/content/drive/My Drive/InteliCipes/RAW_interactions.csv')
recipe_classification = pd.read_csv('/content/drive/My Drive/InteliCipes/recipe_classification.csv')

## Removing null values

In [421]:
recipe.isnull().any()



name               True
id                False
minutes           False
contributor_id    False
submitted         False
tags              False
nutrition         False
n_steps           False
steps             False
description        True
ingredients       False
n_ingredients     False
dtype: bool

In [422]:
interaction.isnull().any()


user_id      False
recipe_id    False
date         False
rating       False
review        True
dtype: bool

In [423]:
recipe_classification.isnull().any()

Unnamed: 0                False
sugar_calories            False
sugar_calories_col        False
fat_sugar_calories        False
fat_sugar_calories_col    False
all_nutrition             False
all_nutrition_col         False
id                        False
dtype: bool

In [424]:
recipe.dropna(axis=0, inplace=True)
interaction.dropna(axis=0, inplace=True)
recipe_classification.dropna(axis=0, inplace=True)

##Remove duplicates

In [425]:
boolean = recipe.duplicated(subset=['id']).any()
boolean

False

In [426]:
boolean = interaction.duplicated(subset=['user_id']).any()
boolean

True

In [427]:
boolean = recipe_classification.duplicated(subset=['id']).any()
boolean

False

In [428]:
recipe = recipe.drop_duplicates(subset=['id'])
recipe

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [429]:
interaction = interaction.drop_duplicates(subset=['user_id'])
interaction

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132358,2000137415,82303,2015-04-13,1,I&#039;ve improvised sauces that turned out be...
1132359,2001898209,82303,2018-07-05,5,This is the best and easiest hot fudge ever. I...
1132360,2002357020,82303,2018-12-05,5,Delicious quick thick chocolate sauce with ing...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...


In [430]:
recipe_classification = recipe_classification.drop_duplicates(subset=['id'])
recipe_classification

,Unnamed: 0,sugar_calories,sugar_calories_col,fat_sugar_calories,fat_sugar_calories_col,all_nutrition,all_nutrition_col,id
0,126987,4,red,1,red,0,red,38
1,23695,3,purple,5,green,6,gray,39
2,22701,0,red,1,red,0,red,40
3,36412,0,red,0,red,4,orange,41
4,22217,0,red,0,red,4,orange,43
...,...,...,...,...,...,...,...,...
226649,13104,4,red,1,red,0,red,537459
226650,1547,4,red,1,red,0,red,537485
226651,137661,7,purple,5,green,6,gray,537543
226652,145532,4,red,1,red,0,red,537671


##Spliting nutrition (N1)

In [431]:
nutrition = recipe.loc[:,['id','nutrition']]
nutrition

,id,nutrition
0,137739,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]"
1,31490,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]"
2,112140,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]"
3,59389,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]"
4,44061,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]"
...,...,...
231632,486161,"[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]"
231633,493372,"[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]"
231634,308080,"[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]"
231635,298512,"[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]"


In [432]:
nutrition['nutrition'] = nutrition['nutrition'].str.extract('\[(.*?)\]', expand=False)
nutrition

,id,nutrition
0,137739,"51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0"
1,31490,"173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0"
2,112140,"269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0"
3,59389,"368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0"
4,44061,"352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0"
...,...,...
231632,486161,"415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0"
231633,493372,"14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0"
231634,308080,"59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0"
231635,298512,"188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0"


In [433]:
nutrition_splitted = nutrition["nutrition"].str.split(",")
nutrition_splitted.head()

0         [51.5,  0.0,  13.0,  0.0,  2.0,  0.0,  4.0]
1     [173.4,  18.0,  0.0,  17.0,  22.0,  35.0,  1.0]
2    [269.8,  22.0,  32.0,  48.0,  39.0,  27.0,  5.0]
3     [368.1,  17.0,  10.0,  2.0,  14.0,  8.0,  20.0]
4     [352.9,  1.0,  337.0,  23.0,  3.0,  0.0,  28.0]
Name: nutrition, dtype: object

In [434]:
nutrition["calories"] = nutrition_splitted.str.get(0)
nutrition["total_fat"] = nutrition_splitted.str.get(1)
nutrition["sugar"] = nutrition_splitted.str.get(2)
nutrition["sodium"] = nutrition_splitted.str.get(4)
nutrition["protein"] = nutrition_splitted.str.get(5)
nutrition["saturated_fat"] = nutrition_splitted.str.get(6)
nutrition

,id,nutrition,calories,total_fat,sugar,sodium,protein,saturated_fat
0,137739,"51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0",51.5,0.0,13.0,2.0,0.0,4.0
1,31490,"173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0",173.4,18.0,0.0,22.0,35.0,1.0
2,112140,"269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0",269.8,22.0,32.0,39.0,27.0,5.0
3,59389,"368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0",368.1,17.0,10.0,14.0,8.0,20.0
4,44061,"352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0",352.9,1.0,337.0,3.0,0.0,28.0
...,...,...,...,...,...,...,...,...
231632,486161,"415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0",415.2,26.0,34.0,44.0,21.0,15.0
231633,493372,"14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0",14.8,0.0,2.0,1.0,0.0,1.0
231634,308080,"59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0",59.2,6.0,2.0,6.0,5.0,0.0
231635,298512,"188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0",188.0,11.0,57.0,7.0,21.0,9.0


In [435]:
nutrition.calories=nutrition.calories.astype('float32')
nutrition.total_fat=nutrition.total_fat.astype('float32')
nutrition.sugar=nutrition.sugar.astype('float32')
nutrition.sodium=nutrition.sodium.astype('float32')
nutrition.protein=nutrition.protein.astype('float32')
nutrition.saturated_fat=nutrition.saturated_fat.astype('float32')
nutrition.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226657 entries, 0 to 231636
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             226657 non-null  int64  
 1   nutrition      226657 non-null  object 
 2   calories       226657 non-null  float32
 3   total_fat      226657 non-null  float32
 4   sugar          226657 non-null  float32
 5   sodium         226657 non-null  float32
 6   protein        226657 non-null  float32
 7   saturated_fat  226657 non-null  float32
dtypes: float32(6), int64(1), object(1)
memory usage: 10.4+ MB


## Removing columns that will not serve

In [436]:
other_columns = recipe.loc[:,['id','nutrition', 'tags', 'ingredients', 'steps', 'contributor_id']]
other_columns

,id,nutrition,tags,ingredients,steps,contributor_id
0,137739,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]","['60-minutes-or-less', 'time-to-make', 'course...","['winter squash', 'mexican seasoning', 'mixed ...","['make a choice and proceed with recipe', 'dep...",47892
1,31490,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]","['30-minutes-or-less', 'time-to-make', 'course...","['prepared pizza crust', 'sausage patty', 'egg...","['preheat oven to 425 degrees f', 'press dough...",26278
2,112140,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]","['time-to-make', 'course', 'preparation', 'mai...","['ground beef', 'yellow onions', 'diced tomato...","['brown ground beef in large pot', 'add choppe...",196586
3,59389,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]","['60-minutes-or-less', 'time-to-make', 'course...","['spreadable cheese with garlic and herbs', 'n...",['place potatoes in a large pot of lightly sal...,68585
4,44061,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]","['weeknight', 'time-to-make', 'course', 'main-...","['tomato juice', 'apple cider vinegar', 'sugar...",['mix all ingredients& boil for 2 1 / 2 hours ...,41706
...,...,...,...,...,...,...
231632,486161,"[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]","['ham', '60-minutes-or-less', 'time-to-make', ...","['celery', 'onion', 'green sweet pepper', 'gar...","['heat oil in a 4-quart dutch oven', 'add cele...",227978
231633,493372,"[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]","['15-minutes-or-less', 'time-to-make', 'course...","['paprika', 'salt', 'garlic powder', 'onion po...",['mix all ingredients together thoroughly'],1500678
231634,308080,"[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]","['60-minutes-or-less', 'time-to-make', 'course...","['hard-cooked eggs', 'mayonnaise', 'dijon must...","['in a bowl , combine the mashed yolks and may...",37779
231635,298512,"[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]","['30-minutes-or-less', 'time-to-make', 'course...","['butter', 'eagle brand condensed milk', 'ligh...",['place melted butter in a large mixing bowl a...,506822


In [437]:
recipe=recipe.drop(columns=['nutrition', 'tags', 'ingredients', 'steps', 'submitted'])
print(recipe.columns)

Index(['name', 'id', 'minutes', 'contributor_id', 'n_steps', 'description',
       'n_ingredients'],
      dtype='object')


In [438]:
nutrition=nutrition.drop(columns=['nutrition',])
print(nutrition.columns) 

Index(['id', 'calories', 'total_fat', 'sugar', 'sodium', 'protein',
       'saturated_fat'],
      dtype='object')


In [439]:
recipe_classification = recipe_classification.drop(columns=['Unnamed: 0','sugar_calories',	'sugar_calories_col','all_nutrition',	'all_nutrition_col' ])
print(recipe_classification.columns) 

Index(['fat_sugar_calories', 'fat_sugar_calories_col', 'id'], dtype='object')


## Rename columns

In [440]:
recipe.rename(columns={'name': 'name_recipe', 'description': 'description_recipe' }, inplace=True)
print(recipe.columns)

Index(['name_recipe', 'id', 'minutes', 'contributor_id', 'n_steps',
       'description_recipe', 'n_ingredients'],
      dtype='object')


In [441]:
nutrition.rename(columns={'id': 'recipe_id'}, inplace=True)
print(nutrition.columns)

Index(['recipe_id', 'calories', 'total_fat', 'sugar', 'sodium', 'protein',
       'saturated_fat'],
      dtype='object')


In [442]:
recipe_classification.rename(columns={'fat_sugar_calories': 'classification_id', 'fat_sugar_calories_col':'classification_group', 'id':'recipe_id' }, inplace=True)
print(recipe_classification.columns)

Index(['classification_id', 'classification_group', 'recipe_id'], dtype='object')


# Modify type

In [443]:
recipe.id=recipe.id.astype('int32')
recipe.minutes=recipe.minutes.astype('int32')
recipe.n_steps=recipe.n_steps.astype('int32')
recipe.n_ingredients=recipe.n_ingredients.astype('int32')
recipe.name_recipe=recipe.name_recipe.astype('string')
recipe.description_recipe=recipe.description_recipe.astype('string')

In [444]:
recipe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226657 entries, 0 to 231636
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   name_recipe         226657 non-null  string
 1   id                  226657 non-null  int32 
 2   minutes             226657 non-null  int32 
 3   contributor_id      226657 non-null  int64 
 4   n_steps             226657 non-null  int32 
 5   description_recipe  226657 non-null  string
 6   n_ingredients       226657 non-null  int32 
dtypes: int32(4), int64(1), string(2)
memory usage: 10.4 MB


#Sort columns


In [445]:
inserted_cols = ['id', 'name_recipe', 'minutes','n_steps','n_ingredients','description_recipe' ]
cols = ([col for col in inserted_cols if col in data] 
        + [col for col in data if col not in inserted_cols])
data = data[cols]

In [446]:
data

,id,minutes,n_steps,n_ingredients,name,contributor_id,submitted,tags,nutrition,steps,description,ingredients
0,137739,55,11,7,arriba baked winter squash mexican style,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]","['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ..."
1,31490,30,9,6,a bit different breakfast pizza,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]","['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg..."
2,112140,130,6,13,all in the kitchen chili,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]","['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato..."
3,59389,45,11,11,alouette potatoes,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n..."
4,44061,190,5,8,amish tomato ketchup for canning,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar..."
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,486161,60,7,22,zydeco soup,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]","['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar..."
231633,493372,5,1,13,zydeco spice mix,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po..."
231634,308080,40,7,8,zydeco ya ya deviled eggs,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]","['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must..."
231635,298512,29,9,10,cookies by design cookies on a stick,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh..."


In [447]:
inserted_cols = ['recipe_id', 'classification_id', 'classification_group' ]
cols = ([col for col in inserted_cols if col in recipe_classification] 
        + [col for col in recipe_classification if col not in inserted_cols])
recipe_classification = recipe_classification[cols]
recipe_classification

,recipe_id,classification_id,classification_group
0,38,1,red
1,39,5,green
2,40,1,red
3,41,0,red
4,43,0,red
...,...,...,...
226649,537459,1,red
226650,537485,1,red
226651,537543,5,green
226652,537671,1,red


# Download csv



In [331]:
from google.colab import files
recipes.to_csv('recipes.csv') 
files.download('recipes.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [449]:
from google.colab import files
interaction.to_csv('interaction.csv') 
files.download('interaction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [450]:
from google.colab import files
recipe_classification.to_csv('recipe_classification.csv') 
files.download('recipe_classification.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [451]:
from google.colab import files
nutrition.to_csv('nutrition.csv') 
files.download('nutrition.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from google.colab import files
# other_columns.to_csv('recipes_other_columns.csv') 
# files.download('recipes_other_columns.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>